In [1]:
import cvxopt
import csv
import pandas as pd
import numpy as np
from random import randint
import time
from tqdm import tqdm, tqdm_notebook
import math
import pickle
from cvxopt import solvers
from cvxopt import matrix

In [122]:
train_data= pd.read_csv('fashion_mnist/train.csv',header=None).to_numpy()
t = np.array([train_data[i,:] for i in range(train_data.shape[0]) if train_data[i,-1]==4 or train_data[i,-1]==5 ] )

In [143]:
t.shape

(4500, 785)

In [144]:
X = t[:, :-1]/255
m= X.shape[0]
Y1 = t[:,-1].reshape((m,1))
Y = 4.5-Y1
Y

array([[ 0.],
       [-1.],
       [-1.],
       ...,
       [ 0.],
       [ 0.],
       [-1.]])

In [145]:
s= time.time()
XX = np.dot(X, X.T)
YY = Y@Y.T
print(time.time()-s)

0.3381187915802002


In [146]:
s= time.time()
P=matrix(np.multiply(XX,YY))
q=matrix(np.ones([m,1])*-1)
print(time.time()-s)
print(YY.shape)

0.17952966690063477
(4500, 4500)


In [147]:
s= time.time()
I = np.identity(m)
I_neg = np.identity(m)*-1
G = matrix(np.concatenate([I,I_neg]))
print(time.time()-s)
# print(I_neg)
A=matrix(Y.reshape((1,m)))
b = matrix([0], tc='d')
Y=Y.reshape(m)
print(A.size, q.size, G.size, h.size,P.size)

0.42592620849609375
(1, 4500) (4500, 1) (9000, 4500) (9000, 1) (4500, 4500)


In [148]:
h = matrix(np.concatenate((np.ones([m,1]),np.zeros([m,1]))))
h.size

(9000, 1)

In [149]:
s= time.time()
sol = solvers.qp(P,q,G,h,A,b)
print(time.time()-s)

     pcost       dcost       gap    pres   dres
 0: -2.2500e+03 -6.7500e+03  2e+04  1e+00  9e-16
 1: -1.8720e+03 -3.6392e+03  2e+03  1e-02  4e-16
 2: -2.2100e+03 -2.2828e+03  8e+01  6e-04  4e-16
 3: -2.2496e+03 -2.2503e+03  8e-01  6e-06  4e-16
 4: -2.2500e+03 -2.2500e+03  8e-03  6e-08  3e-16
 5: -2.2500e+03 -2.2500e+03  8e-05  6e-10  3e-16
Optimal solution found.
22.755807161331177


In [150]:
alpha=np.array(sol['x'])
print(sol['status'])
tt =[i for i in alpha if i >=0.1]
alpha.max()

optimal


0.9999999822128603